In [1]:
train_dir='data/dogs_vs_cats/train'
test_dir='data/dogs_vs_cats/test'

In [2]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['KMP_DUPLICATE_OK'] = "TRUE"

In [3]:
model = models.Sequential() #padding안함 + input image 작음
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3))) #32*9*3+32
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) #32*64*9+64
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu')) #64*128*9+128
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu')) #128*128*9+ 128
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten()) #7*7*128 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu')) #6272*512+512
model.add(layers.Dense(1, activation='sigmoid')) #512*1+1
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [4]:
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿈
        target_size=(150, 150),
        batch_size=20,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

#label 0,1 만들어줌
#target size로 작게하거나 크게 함
#이진 분류: 'binary', 다중분류: 'categorical'

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [8]:
model_path = './dogs-vs-cats.hdf5'
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

In [10]:
history = model.fit(
      train_generator,
      epochs=50,
      validation_data=validation_generator,
      callbacks=[early_stopping, checkpoint])

Epoch 1/50
1000/1000 [==============================] - ETA: 0s - loss: 0.5948 - accuracy: 0.6786
Epoch 1: val_loss improved from inf to 0.57575, saving model to .\dogs-vs-cats.hdf5
1000/1000 [==============================] - 267s 267ms/step - loss: 0.5948 - accuracy: 0.6786 - val_loss: 0.5758 - val_accuracy: 0.7124
Epoch 2/50
1000/1000 [==============================] - ETA: 0s - loss: 0.5083 - accuracy: 0.7475
Epoch 2: val_loss improved from 0.57575 to 0.45642, saving model to .\dogs-vs-cats.hdf5
1000/1000 [==============================] - 215s 215ms/step - loss: 0.5083 - accuracy: 0.7475 - val_loss: 0.4564 - val_accuracy: 0.7934
Epoch 3/50
1000/1000 [==============================] - ETA: 0s - loss: 0.4253 - accuracy: 0.8062
Epoch 3: val_loss improved from 0.45642 to 0.41250, saving model to .\dogs-vs-cats.hdf5
1000/1000 [==============================] - 215s 215ms/step - loss: 0.4253 - accuracy: 0.8062 - val_loss: 0.4125 - val_accuracy: 0.8262
Epoch 4/50
1000/1000 [=============